In [3]:
from pydrake.all import GraphOfConvexSets, HPolyhedron, Hyperrectangle, ConvexSet
import numpy as np
import typing as T
from scipy.linalg import block_diag

In [4]:
def concatenate_polyhedra(sets:T.List[ConvexSet]) -> HPolyhedron:
    res = None
    for cset in sets:
        assert isinstance(cset, Hyperrectangle) or isinstance(cset, HPolyhedron)
        if isinstance(cset, Hyperrectangle):
            hpoly = cset.MakeHPolyhedron()
        else:
            hpoly = cset

        if res is None:
            res = hpoly
        else:
            new_A = block_diag(res.A(), hpoly.A())
            new_b = np.vstack((res.b().reshape((len(res.b()),1)), hpoly.b().reshape((len(hpoly.b()),1))))
            res = HPolyhedron(new_A, new_b)
    return res

In [11]:
regions = [ Hyperrectangle([0,0], [2,2]), Hyperrectangle([1,1], [4,3]) ]

gcs = GraphOfConvexSets()
v_dict = dict() # type: T.Dict[str, GraphOfConvexSets.Vertex]

# NOTE: you might wanna call reduce inequalities on the regions somewhere

# add vertices
for i, region in enumerate(regions):
    # each convex set is a segment
    name = "v_"+str(i)
    v = gcs.AddVertex(concatenate_polyhedra([region, region]), name)
    v_dict[name] = v

    # TODO: add some cost here or on edges
    v.AddCost(v.x().dot(v.x()))


# add edges:
for i, region_i in enumerate(regions):
    # each convex set is a segment
    for j in range(i+1, len(regions)):
        region_j = regions[j]
        # check if regions intersect
        if region_i.IntersectsWith(region_j):
            # add an edge
            v_i = v_dict["v_"+str(i)]
            v_j = v_dict["v_"+str(j)]

            e_ij = gcs.AddEdge(v_i,v_j)
            # NOTE: i'm adding bidirectional edges here. 
            # this may not be necessary depending on your application
            e_ji = gcs.AddEdge(v_j,v_i)

            # add edge constraint: points must coincide
            assert len(e_ij.xu())//2 == len(e_ij.xv())//2, "sets don't have same dimension"
            n = len(e_ij.xu())//2
            for i in range(n):
                e_ij.AddConstraint(e_ij.xu()[n+i] == e_ij.xv()[i])
                e_ji.AddConstraint(e_ji.xu()[n+i] == e_ji.xv()[i])

            # TODO: add some cost here or on vertices


# TODO: add some initial constraints
s_point = [0,0]
t_point = [4,3]

# NOTE: initial point at the first region is s_point
vs = v_dict["v_0"]
n = len(e_ij.xu())//2
for i in range(n):
    vs.AddConstraint(vs.x()[i] == s_point[i])


# NOTE: final point at the last region is t_point
vt = v_dict["v_"+str(len(regions)-1)]
n = len(e_ij.xu())//2
for i in range(n):
    vt.AddConstraint(vt.x()[n+i] == t_point[i])

solution = gcs.SolveShortestPath(vs, vt)
assert solution.is_success()


INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.


In [18]:
solution_edges = gcs.GetSolutionPath(vs,vt,solution)
solution_vertices = [edge.u() for edge in solution_edges] + [solution_edges[-1].v()]
segments = [solution.GetSolution(v.x()) for v in solution_vertices]
points = [segment[:len(segment)//2] for segment in segments] + [ segments[-1][len(segments[-1])//2:] ]
points

[array([0., 0.]), array([1., 1.]), array([4., 3.])]